In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
from shapely.geometry import Point

import plotly.express as px
import plotly.graph_objects as go

import ipywidgets as widgets

In [ ]:
mapbox_access_token = open('Resources/token.mapbox_token').read()
px.set_mapbox_access_token(mapbox_access_token)

In [ ]:
df = pd.read_csv('Resources/Anabasis.csv', keep_default_na=False)
df['geometry'] = gpd.points_from_xy(df['longitude'], df['latitude'])

gdf = gpd.GeoDataFrame(df)
gdf.set_crs(epsg=4326, allow_override=True, inplace=True)

In [ ]:
start_date = np.datetime64('-401-04-20T00:00:00')

def generate_date(date):
    return start_date + np.timedelta64(date, 'D')

gdf['start_date'] = gdf.apply(lambda row: generate_date(row['start_day']), axis = 1)
gdf['end_date'] = gdf.apply(lambda row: generate_date(row['end_day']), axis = 1)

In [ ]:
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

def prettifyDate(date):
    dateString = np.datetime_as_string(date, unit='D')
    
    isBC = dateString[0] == '-'
    if isBC:
        dateString = dateString[1:]
    
    parts = dateString.split('-')
    
    result = parts[2] + " " + months[int(parts[1]) - 1] + " " + parts[0]
    if isBC:
        result += " BC"
    
    return result

In [ ]:
marker_lookup = {
    "city": "castle",
    "battle": "cross",
    "harbour": "harbor",
    "camp": "campsite"
}
    
def get_marker_symbol(event_type):
    return marker_lookup.get(event_type)

def get_travel_symbol():
    return "horse-riding"

gdf['marker'] = gdf.apply(lambda row: get_marker_symbol(row['event_type']), axis = 1)

In [ ]:
fig = go.Figure(
    go.Scattermapbox(
        lat=gdf.geometry.y,
        lon=gdf.geometry.x,
        mode='markers+lines',
        marker_symbol=gdf.marker,
        marker_size=15,
        marker_color="white",
        customdata=np.stack((gdf['event_name'], gdf['start_date'].apply(prettifyDate), gdf['end_date'].apply(prettifyDate), gdf['event_description'], gdf['bibliography']), axis=-1),
        hovertemplate = '<b>%{customdata[0]}</b><br><br>' +
            'Start: %{customdata[1]}<br>'+
            'End: %{customdata[2]}<br>'+
            '%{customdata[3]}<br>'+
            'Bibliography: %{customdata[4]}'+
            '<extra></extra>'
    )
)

In [ ]:
extents_latitude =[gdf["latitude"].min(), gdf["latitude"].max()]
extents_longitude =[gdf["longitude"].min(), gdf["longitude"].max()]
max_bound = max(abs(extents_latitude[0]-extents_latitude[1]), abs(extents_longitude[0]-extents_longitude[1])) * 111
zoom = 12.5 - np.log(max_bound)

fig.update_layout(
    title='Anabasis - March of the Ten Thousand',
    autosize=False,
    width=1000,
    height=750,
    hovermode='closest',
    mapbox=dict(
        accesstoken = mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=extents_latitude[0] + (extents_latitude[1] - extents_latitude[0]) / 2,
            lon=extents_longitude[0] + (extents_longitude[1] - extents_longitude[0]) / 2
        ),
        pitch=0,
        zoom=zoom,
        style='mapbox://styles/wolololf/ckjcyusqlks2a19qts3en5dnf'
    ))

In [ ]:
days = range(gdf['end_day'].max() + 1)

frames = []

for day in days:
    filtered_gdf = gdf[gdf['start_day'] <= day]

    idx = filtered_gdf['start_day'].idxmax()
    previous_day = filtered_gdf.iloc[idx]

    if previous_day['end_day'] < day:
        next_day = gdf.iloc[idx+1]

        date = generate_date(day)

        departure_day = previous_day['end_day']
        arrival_day = next_day['start_day']
        factor = (day - departure_day) / (arrival_day - departure_day)

        lat = previous_day['latitude'] + (next_day['latitude'] - previous_day['latitude']) * factor
        lon = previous_day['longitude'] + (next_day['longitude'] - previous_day['longitude']) * factor

        interpolated = {
            'event_name': "On the road",
            'event_description': "",
            'bibliography': "",
            'start_day': day,
            'start_date': date,
            'end_day': day,
            'end_date': date,
            'latitude': lat,
            'longitude': lon,
            'geometry': Point(lon, lat),
            'marker': get_travel_symbol()
        }

        filtered_gdf = filtered_gdf.append(interpolated, ignore_index=True)
    
    frame = go.Frame(
        data=[
            go.Scattermapbox(
            lat=filtered_gdf.geometry.y,
            lon=filtered_gdf.geometry.x,
            marker_symbol=filtered_gdf.marker,
            customdata=np.stack(
                (
                    filtered_gdf['event_name'],
                    filtered_gdf['start_date'].apply(prettifyDate),
                    filtered_gdf['end_date'].apply(prettifyDate),
                    filtered_gdf['event_description'],
                    filtered_gdf['bibliography']),
                    axis=-1
                )
            )
        ], 
        traces= [0],
        name=day
    )
    
    frames.append(frame)

fig.update(frames=frames)

In [ ]:
transition = dict(
    duration=100,
    easing = "cubic-in-out"
)
frame_anim = dict(
    duration=100,
    redraw=True
)

sliders = dict(steps=[],
    active=0,
    currentvalue=dict(
        font=dict(size=12), 
        prefix='Date: ', 
        visible=True, 
        xanchor= 'left'
    ),
    transition=transition,
    x=0.15,
    y=0,
    len=0.85,
    xanchor="left",
    yanchor="top"
)

for day in days:
    slider_step = dict(
        method='animate',
        value=day,
        label=prettifyDate(generate_date(day)),
        args=[
            [day],
            dict(
                mode='immediate',
                frame=frame_anim,
                transition=transition
            )
        ]
    )
    
    sliders["steps"].append(slider_step)

fig.update_layout(sliders=[sliders])

In [ ]:
updatemenus = dict(
    type='buttons',
    showactive=True,
    x=0,
    y=-0.05,
    xanchor = "left",
    yanchor = "top",
    direction = "left",
    buttons=[
        dict(
            label='Play',
            method='animate',
            args=[
                None, 
                dict(
                    frame=frame_anim,
                    transition=transition,
                    fromcurrent=True,
                    mode='immediate'
                )
            ]
        ),
        dict(
            label='Pause',
            method='animate',
            args=[
                [None], 
                dict(
                    frame=frame_anim,
                    transition=transition,
                    mode='immediate'
                )
            ]
        )
    ]
)

fig.update_layout(updatemenus=[updatemenus])

In [ ]:
fig.show()